<a href="https://colab.research.google.com/github/JP-SystemsX/24cast/blob/master/AI_Grid_Hackathon_JP_DEV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
!pip install -q wget
!pip install -q autogluon
!pip install -q pyarrow==15.0.2
!pip install -q pandas

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 19.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.8/234.8 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.4/63.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 428.0/428.0 kB 28.0 MB/

In [2]:
!wget https://zenodo.org/records/3899018/files/LoadProfile_20IPs_2016.csv -q
!wget https://zenodo.org/records/3899018/files/LoadProfile_30IPs_2017.csv -q

In [3]:
!ls

LoadProfile_20IPs_2016.csv  LoadProfile_30IPs_2017.csv	sample_data


In [4]:
import pandas as pd
from copy import deepcopy
import numpy as np
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

In [66]:
prediction_length = 48
long_runtime = (3*60-5)*60 # in seconds
dev_runtime = 5*60 # in seconds

train_1 = pd.read_csv("tune/2016_train.csv")
train_2 = pd.read_csv("tune/2017_train.csv")
val_1 = pd.read_csv("tune/2016_val.csv")
val_2 = pd.read_csv("tune/2017_val.csv")

def prepare_AG_data(dfs: list[pd.DataFrame]):
    # Prepare Data
    melted_dfs = []
    for df in dfs:
        df = deepcopy(df)#[["Time stamp", "LG 1"]]
        df.loc[:, ["Time stamp"]] = df["Time stamp"].str.replace(r'[^0-9.: ]', '', regex=True).str.strip()
        df.loc[:, ["Time stamp"]] = pd.to_datetime(df["Time stamp"])
        # TODO Normalize Data else MASE means nothing
        # Concat Columns & Assign Ids
        df = df.melt(id_vars=['Time stamp'], var_name='item_id', value_name='target')
        melted_dfs.append(df)
    df = pd.concat(melted_dfs)
    df = TimeSeriesDataFrame.from_data_frame(
        df,
        id_column="item_id",
        timestamp_column="Time stamp"
    )
    return df

train = prepare_AG_data([train_1, train_2])
val = prepare_AG_data([val_1, val_2])
train

target
item_id timestamp                  
LG 1    2016-01-01 00:15:00    39.0
        2016-01-01 00:30:00    41.0
        2016-01-01 00:45:00    39.0
        2016-01-01 01:00:00    40.0
        2016-01-01 01:15:00    39.0
...                             ...
LG 22   2017-08-31 23:00:00     1.6
        2017-08-31 23:15:00     1.4
        2017-08-31 23:30:00     1.0
        2017-08-31 23:45:00     0.7
        2018-01-01 00:00:00     NaN

[817728 rows x 1 columns]

In [61]:
%%time

predictor = TimeSeriesPredictor(
    prediction_length=prediction_length,
    path="autogluon-m4-hourly",
    target="target",
    eval_metric="MAPE",
    freq="15min"
)

# Dev Setting
predictor = predictor.fit(
    train,
    presets="medium_quality",
    time_limit=dev_runtime,
)

Beginning AutoGluon training... Time limit = 300s
AutoGluon will save models to 'autogluon-m4-hourly'
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.9.13
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22631
CPU Count:          12
GPU Count:          0
Memory Avail:       1.16 GB / 15.34 GB (7.5%)
Disk Space Avail:   94.20 GB / 474.72 GB (19.8%)
Setting presets to: medium_quality

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': MAPE,
 'freq': '15min',
 'hyperparameters': 'light',
 'known_covariates_names': [],
 'num_val_windows': 1,
 'prediction_length': 48,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': False,
 'target': 'target',
 'time_limit': 300,
 'verbosity': 2}

train_data with frequency 'None' has been resampled to frequency '15min'.
Provided train_data has 12276

CPU times: total: 10min 59s
Wall time: 3min 42s


In [62]:
predictor.evaluate(val)

data with frequency 'None' has been resampled to frequency '15min'.
Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble


{'MAPE': -0.23422281837336656}

In [63]:
predictor.leaderboard()

,model,score_val,pred_time_val,fit_time_marginal,fit_order
0,WeightedEnsemble,-0.641658,5.538226,1.143834,8
1,TemporalFusionTransformer,-1.003017,0.828575,71.082603,7
2,RecursiveTabular,-2.269861,4.709651,23.982011,3
3,Theta,-2.335450,35.483332,1.388065,6
4,ETS,-2.335450,38.069725,1.158395,5
5,DirectTabular,-4.192494,2.914106,21.878970,4
6,SeasonalNaive,-8.322310,0.299053,1.236411,2
7,Naive,-8.322310,7.965436,2.612299,1


# Level UP

In [65]:
%%time

predictor = TimeSeriesPredictor(
    prediction_length=prediction_length,
    path="autogluon-m4-hourly",
    target="target",
    eval_metric="MAPE",
    freq="15min"
)

# TODO Increase Time Limit For Final Version
predictor = predictor.fit(
    train,
    presets="best_quality",
    time_limit=long_runtime,
)

Beginning AutoGluon training... Time limit = 1200s
AutoGluon will save models to 'autogluon-m4-hourly'
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.9.13
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22631
CPU Count:          12
GPU Count:          0
Memory Avail:       2.54 GB / 15.34 GB (16.6%)
Disk Space Avail:   95.84 GB / 474.72 GB (20.2%)
Setting presets to: best_quality

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': MAPE,
 'freq': '15min',
 'hyperparameters': 'default',
 'known_covariates_names': [],
 'num_val_windows': 2,
 'prediction_length': 48,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': False,
 'target': 'target',
 'time_limit': 1200,
 'verbosity': 2}

train_data with frequency 'None' has been resampled to frequency '15min'.
Provided train_data has 12

CPU times: total: 1h 42min 26s
Wall time: 15min 41s


In [67]:
print(predictor.evaluate(val))
predictor.leaderboard()

data with frequency 'None' has been resampled to frequency '15min'.
Model not specified in predict, will default to the model with the best validation score: SeasonalNaive


{'MAPE': -0.5134284752677395}


,model,score_val,pred_time_val,fit_time_marginal,fit_order
0,TemporalFusionTransformer,NaN,1.105947,230.919984,9
1,SeasonalNaive,NaN,0.230238,5.161949,1
2,RecursiveTabular,NaN,4.860324,92.884815,2
3,PatchTST,NaN,0.527223,233.131572,11
4,NPTS,NaN,0.234210,2.296155,5
5,DynamicOptimizedTheta,NaN,0.948643,36.317193,6
6,DirectTabular,NaN,2.703185,37.566162,3
7,DeepAR,NaN,3.402964,233.758418,10
8,CrostonSBA,NaN,0.375515,16.903844,4
9,AutoETS,NaN,1.180979,20.756185,7


## Full Search

In [ ]:
# Model Exploration Exploration 
# TODO Custom Config that compares all Models from Model Zoo
predictor = TimeSeriesPredictor(prediction_length=prediction_length)
predictor = predictor.fit(
    train,
    hyperparameters={
        "Chronos": {
            "model_path": "tiny",
            "batch_size": 64,
            "device": "cpu",
        },
        "NaiveModel": {},
        "SeasonalNaiveModel": {},
        "AverageModel": {},
        "SeasonalAverageModel": {},
        "ZeroModel": {},
        "ETSModel": {},
        "AutoARIMAModel": {},
        "AutoETSModel": {},
        "AutoCESModel": {},
        "ThetaModel": {},
        "ADIDAModel": {},
        "CrostonClassicModel": {},
        "CrostonOptimizedModel": {},
        "CrostonSBAModel": {},
        "IMAPAModel": {},
        "NPTSModel": {},
        "DeepARModel": {},
        "DLinearModel": {},
        "PatchTSTModel": {},
        "SimpleFeedForwardModel": {},
        "TemporalFusionTransformerModel": {},
        "TiDEModel": {},
        "WaveNetModel": {},
        "DirectTabularModel": {},
        "RecursiveTabularModel": {},
    },
    # skip running backtests during `fit`, as only consider 1 model
    skip_model_selection=True, 
    verbosity=0,
)
# TODO Train for a few hours (with GPU)
predictor = predictor.fit(
    train,
    presets="best_quality",
    time_limit=long_runtime, # Colab Timeout
)

# TODO Investigate Leaderboard
print(predictor.evaluate(val))
predictor.leaderboard()

In [ ]:
# TODO Tune best found Config

# Chronos

In [ ]:
# TODO Test out Tiny Chronos compare to Large Chronos -> GPU and delay Worth it
# TODO Is Finetuned Tiny much better than untuned Tiny
# TODO Speed up Chronos
# TODO How compare to classical AG